In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import os, shutil
from sklearn.model_selection import train_test_split

In [3]:
image_path="./rps-cv-images"

#### removing corrupted_images
for category in os.listdir(folder_path):

*  It loops through each subfolder in the main folder (e.g., rock, paper, scissors).

for img_file in os.listdir(category_path):

*  It then loops through every image in that subfolder.

img = Image.open(img_path)

*  Tries to open the image using Pillow (PIL). This checks if the image can be read.

img.verify()

*  This verifies the image is not corrupted or partially downloaded. It does not load the image into memory (it’s lightweight).

If it fails (IOError or SyntaxError)

*   That means the image is likely corrupted (damaged file, wrong format, etc.).

*   It prints the path of the bad image and deletes it using os.remove(img_path).

In [4]:
def remove_corrupted_images(image_path):
    for category in os.listdir(image_path):
        category_path = os.path.join(image_path, category)
        for img_file in os.listdir(category_path):
            img_path = os.path.join(category_path, img_file)
            try:
                img = Image.open(img_path)
                img.verify()
            except (IOError, SyntaxError) as e:
                print("Removing corrupted image:", img_path)
                os.remove(img_path)


In [6]:
remove_corrupted_images("rps-cv-images")

#### Loading the data 
Load all the images (rock, paper, scissors) from folders, resize them to 64×64 pixels, normalize their pixel values, automatically assign labels based on folder names, and split them into training and validation sets.
rescale=1./255 → Scales image pixel values from range [0, 255] to [0, 1] (which helps the neural network train better).
validation_split=0.2 → Splits your dataset: 80% for training, 20% for validation
##### loading the training data 
'rps-cv-images' → Path to the main folder that contains the rock/, paper/, and scissors/ folders.
target_size=(64, 64) → Resizes all images to 64×64 pixels.
batch_size=32 → Loads 32 images at a time (good for memory efficiency).
class_mode='categorical' → Labels are one-hot encoded. So if the classes are rock, paper, scissors, each image label looks like:

Rock → [1, 0, 0]

Paper → [0, 1, 0]

Scissors → [0, 0, 1]
subset='training' → This line tells it to use 80% of the data for training (from the split defined above).

##### loading the validation data 
It uses the remaining 20% of images (subset='validation').

In [7]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
)

train_generator = datagen.flow_from_directory(
    image_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode="categorical",
    subset="training",
)

val_generator = datagen.flow_from_directory(
    image_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode="categorical",
    subset="validation",
)

Found 1751 images belonging to 3 classes.
Found 437 images belonging to 3 classes.


In [9]:
src = 'rps-cv-images'
dst = 'rps-split'
splits = ['train', 'val', 'test']

for label in os.listdir(src):
    files = os.listdir(os.path.join(src, label))
    train, temp = train_test_split(files, test_size=0.3, random_state=42)
    val, test = train_test_split(temp, test_size=0.5, random_state=42)
    
    for split, split_list in zip(splits, [train, val, test]):
        os.makedirs(os.path.join(dst, split, label), exist_ok=True)
        for f in split_list:
            shutil.copy(
                os.path.join(src, label, f),
                os.path.join(dst, split, label, f)
            )


In [10]:
train_gen = datagen.flow_from_directory('rps-split/train', target_size=(64, 64), class_mode='categorical')
val_gen = datagen.flow_from_directory('rps-split/val', target_size=(64, 64), class_mode='categorical')
test_gen = datagen.flow_from_directory('rps-split/test', target_size=(64, 64), class_mode='categorical', shuffle=False)


Found 1531 images belonging to 3 classes.
Found 328 images belonging to 3 classes.
Found 329 images belonging to 3 classes.


cnn model in PyTorch This model will be fixed and used for all optimizer experiments.

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 16 * 16, 128)  # Assuming input is 64x64 → after pooling 2x: 16x16
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 3)  # 3 output classes (rock, paper, scissors)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        x = x.view(-1, 64 * 16 * 16)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
